In [1]:
from load_data import load_glove_model

%load_ext autoreload
%autoreload 2

In [2]:
root = "/Users/shikha/UMass/fall2017/NLP/projects"
dataset = root + "/irony/SemEval2018-Task3/datasets"
glove_file = root + "/glove.6B/glove.6B.100d.txt"

# Load glove file
wordsList, wordVectors = load_glove_model(glove_file)


Loading Glove Model...
Done.  400000  words loaded!
[ 0.59368   0.44825   0.5932    0.074134  0.11141   1.2793    0.16656
  0.2407    0.39045   0.32766  -0.75034   0.35007   0.76057   0.38067
  0.17517   0.031791  0.46849  -0.21653  -0.46282   0.39967   0.16623
 -0.011477  0.044059  0.30325   0.6153    0.47047  -0.44036  -1.5963
  0.18433   0.23193   0.20452   0.51617   0.65734  -0.3452    0.23446
 -0.62004  -0.68741   0.28575   1.0605    0.46916  -0.85149   0.10154
  0.21426  -0.20587   0.23636   0.21321  -0.21287   0.12107   0.18766
 -0.23282  -0.25499  -0.39631   0.84379   1.6801   -0.40941  -1.9976
 -0.69868   0.21732   1.2197    0.55126   0.44095   0.72588  -0.092053
 -0.022406  0.72039   0.1076    0.84116   0.30312  -0.42544   0.056362
  0.13109  -0.071181 -0.10579   0.56677   0.54547   0.84113   0.14861
 -0.62628  -0.68391  -1.0831   -0.088385  0.32167   0.47794   0.091868
 -1.2559   -1.2268    0.085401  0.36833   0.081566 -0.76611   0.87751
 -0.22008   0.82401  -0.092207 -0.45

In [3]:
print(len(wordsList))
print(wordVectors.shape)

400000
(400000, 100)


In [4]:
from load_data import load_dataset

# Load dataset
filename = dataset + "/train/abc"
X, Y = load_dataset(filename)
train_len = int(0.8 * X.shape[0])
X_train = X[0:train_len]
Y_train = Y[0:train_len]
X_test = X[train_len:]
Y_test = Y[train_len:]

Loading Dataset...
removing urls from tweets...
Done.  3817  tweets loaded!


In [5]:
import nltk

def tokenize(sentence):
    tokenizer = nltk.TweetTokenizer(strip_handles=True)
    return tokenizer.tokenize(sentence.decode('utf-8'))

In [6]:
# counting average no words in a sentence
count = 0
for sentence in X_train:
    words = tokenize(sentence)
    count += len(words)
avg = count/train_len
print "Average no of words in a sentence = ", avg

Average no of words in a sentence =  15


In [7]:
maxSeqLength = 25

In [8]:
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8

In [9]:
import numpy as np

# ids = np.zeros((X_train.shape[0], maxSeqLength), dtype='int32')
# i = 0
# for sentence in X_train:
#     words = tokenize(sentence)
#     indexCounter = 0
#     for word in words:
#         try:
#             ids[i][indexCounter] = wordsList.index(word)
#         except ValueError:
#             ids[i][indexCounter] = 399999 #Vector for unknown words
#         indexCounter = indexCounter + 1
#         if indexCounter >= maxSeqLength:
#            break
#     i += 1

# np.save('idsMatrix', ids)

In [10]:
ids = np.load('idsMatrix.npy')

In [11]:
batchSize = 100
lstmUnits = 12
numClasses = 2
iterations = 35

In [12]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [13]:
numDimensions = 100
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors, input_data)

In [14]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits, reuse=tf.AUTO_REUSE)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float64)

In [15]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0])-1)
prediction = (tf.matmul(tf.cast(last, tf.float32), weight) + bias)

In [16]:
print prediction.shape
print last.shape, weight.shape, value.get_shape()[0]
print value.shape
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

(100, 2)
(100, 12) (12, 2) 25
(25, 100, 12)


In [17]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [18]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, tf.Session().graph)

In [19]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros((batchSize, maxSeqLength))
    for i in range(batchSize):
        num = randint(1,ids.shape[0]-1)
        if Y_train[num] == 1:
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels


In [20]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

print "Training the model..."
for i in range(iterations):
    #Next Batch of reviews
    print "Iteration: ", i
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

    #Write summary to Tensorboard
    if (i % 10 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

    #Save the network every 10,000 training iterations
    if (i % 10 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

Training the model...
Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
saved to models/pretrained_lstm.ckpt-10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
saved to models/pretrained_lstm.ckpt-20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
saved to models/pretrained_lstm.ckpt-30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34


In [ ]:
# ------------TEST-------------------


# ids_test = np.zeros((X_test.shape[0], maxSeqLength), dtype='int32')
# i = 0
# for sentence in X_test:
#     words = tokenize(sentence)
#     indexCounter = 0
#     for word in words:
#         try:
#             ids_test[i][indexCounter] = wordsList.index(word)
#         except ValueError:
#             ids_test[i][indexCounter] = 399999 #Vector for unknown words
#         indexCounter = indexCounter + 1
#         if indexCounter >= maxSeqLength:
#            break
#     i += 1

# np.save('idsMatrix_Test', ids_test)

In [21]:
ids_test = np.load('idsMatrix_Test.npy')

In [22]:
def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,ids_test.shape[0]-1)
        if Y_test[num] == 1:
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids_test[num-1:num]
    return arr, labels

In [23]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-30


In [24]:
iterations = 10
print ids_test.shape, X_test.shape
acc = []
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch()
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)
#     a = sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})
#     acc.append(a)
#     print("Loss for this batch: ", loss_val)
#     print("Accuracy for this batch: ", a)

# print "Overall accuracy: ", sum(acc)/iterations

(764, 25) (764,)
('Accuracy for this batch:', 50.999999046325684)
('Accuracy for this batch:', 50.999999046325684)
('Accuracy for this batch:', 40.000000596046448)
('Accuracy for this batch:', 52.999997138977051)
('Accuracy for this batch:', 49.000000953674316)
('Accuracy for this batch:', 34.000000357627869)
('Accuracy for this batch:', 46.99999988079071)
('Accuracy for this batch:', 38.999998569488525)
('Accuracy for this batch:', 56.999999284744263)
('Accuracy for this batch:', 49.000000953674316)
